# Loop over some images, interpolating between them

In [7]:
import numpy as np
import random
from collections import deque
import matplotlib.patches as patches
import cartopy.crs as ccrs

import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
from pathlib import Path
import os
from subprocess import run
from matplotlib.colors import LinearSegmentedColormap, to_rgba
import matplotlib.colors as mcolors
from matplotlib.image import imread

In [45]:
# Interpolate using rife, 8x after 4x after trying 2x 
# problem was it didn't overwrite, so clobber all frames before running this 
import subprocess

RIFE = True 
DAIN = False # never did work, lots of back and forth with Claude, maybe Mac file issues?

interpnumbers = [2,3,4,5,6,7]

for i,interpnumber in enumerate(interpnumbers):

    frame_folder = Path('/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest_4frames/')
    NFout = str(interpnumber*4) 
    frame_folder = Path('/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest/')
    NFout = str(interpnumber*14) 
    
    framess_folder=Path('/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest'+\
                        str(interpnumber))
    if(DAIN): 
        framess_folder=Path('/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest_DAIN'+str(interpnumber))

    framess_folder.mkdir(parents=True, exist_ok=True)
    print(framess_folder) 
    
# Run RIFE interpolation to make interleaved frames, uses only two frames and optical flow 
# DAIN blew my memory with 14 frames, try with just 3 perhaps. 

    if(RIFE): 
        subprocess.run([
            "/Users/bmapes/Box/Sky_Symphony_Box/rife-ncnn-vulkan-20221029-macos/rife-ncnn-vulkan",  # or full path to executable
            "-i", frame_folder,
            "-o", framess_folder,
            "-n", NFout,  # number of frames to generate, 4 or 14 times interpnumber 
            "-m", "rife-v4.6"
        ])
        
    if(DAIN):
        result = subprocess.run([
        '/Users/bmapes/Box/Sky_Symphony_Box/dain-ncnn-vulkan-20220728-macos/dain-ncnn-vulkan',
        '-0', str(frame_folder / '00000000.png'),
        '-1', str(frame_folder / '00000001.png'),
        '-o', str(framess_folder / 'test_output.png'),
        '-n', '8'
        ], capture_output=True, text=True)

/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest2


[0 Apple M2]  queueC=0[1]  queueG=0[1]  queueT=0[1]
[0 Apple M2]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[0 Apple M2]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 Apple M2]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1


/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest3


[0 Apple M2]  queueC=0[1]  queueG=0[1]  queueT=0[1]
[0 Apple M2]  bugsbn1=0  bugbilz=0  bugcopc=0  bugihfa=0
[0 Apple M2]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 Apple M2]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1


/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest4


[0 Apple M2]  queueC=0[1]  queueG=0[1]  queueT=0[1]
[0 Apple M2]  bugsbn1=0  bugbilz=97  bugcopc=0  bugihfa=0
[0 Apple M2]  fp16-p/s/a=1/1/1  int8-p/s/a=1/1/1
[0 Apple M2]  subgroup=32  basic=1  vote=1  ballot=1  shuffle=1


KeyboardInterrupt: 

In [46]:
NFout

'56'

Unfortunately, RIFE (and most frame interpolation models) inherently work frame-pair by frame-pair - they estimate optical flow between consecutive frames and can't look ahead or behind to smooth the trajectory.
However, there are a few alternatives that use temporal context:
    
1. DAIN (Depth-Aware Interpolation Network)
Uses depth estimation + multi-frame context for smoother motion. Slower but potentially better for your use case:
bashpip install dain-ncnn-vulkan
    
2. CAIN (Channel Attention Interaction Network)
Also incorporates temporal information across multiple frames rather than just pairs.

# Make a video using ffmpeg

In [16]:
# Then from interpolated frames make video at desired length
# video_filename = "/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/MERGEDIR_BOXES/"+"Irma_bmc_4xframes.mp4"

subprocess.run([
    "ffmpeg", "-y",
    "-framerate", "12",  # 11.67 = 5600 frames / 480 sec for 8 min, I have 5328 frames 
    "-i", f"{framess_folder}/%08d.png",
    "-c:v", "libx264", "-pix_fmt", "yuv420p",
    "/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/Irma_interpolationtest_framerate12.mp4"
])

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with clang version 18.1.8
  configuration: --prefix=/Users/bmapes/.local/share/mamba/envs/hk25 --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --enable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libharfbuzz --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1746479731466/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --disable-gnutls --enable-libvpx --enable-libass --enable-pthreads --enable-libopenvino --enable-gpl --enable-libx264 --enable-libx265 --enable-libmp3lame --enable-libaom --enable-libsvtav1 --enable-libxml2 --enable-pic --enable-shared --disable-static --enable-version3 --enable-zli

CompletedProcess(args=['ffmpeg', '-y', '-framerate', '12', '-i', '/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017_weekprior/frames16_interptest5/%08d.png', '-c:v', 'libx264', '-pix_fmt', 'yuv420p', '/Users/bmapes/Box/Sky_Symphony_Box/IRMA2017/Irma_interpolationtest_framerate12.mp4'], returncode=0)